# NLP Final Project

#### Lindsey Feingold, 12/02/2019

My final project compares the Harry Potter books versus the movies. I extracted all of the spoken text per main character and compared the quality of the text between the books to the movies. I first used regular expressions to do this. 

My overall research question is: are the characters speaking differently in the books versus the movies? Some examples of this will be through tf-idf (spoken words per character), word complexity, and sentence length.

Dataset: the Harry Potter R Package on Github has each book text. I can convert the rda of each file to a csv. https://github.com/bradleyboehmke/harrypotter/tree/master/data. For the movies, the scripts are published here: https://www.kaggle.com/gulsahdemiryurek/harry-potter-dataset. I could also use the Harry Potter API for more information if needed: https://www.potterapi.com/

In [1]:
import nltk
import nltk.data
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import xlrd
pd.options.display.max_rows = 999
pd.set_option('max_colwidth', 200)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import fuzzywuzzy as fw
matplotlib.rcParams['pdf.fonttype'] = 42
%matplotlib inline
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from textatistic import Textatistic
from tabulate import tabulate

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lindseyfeingold/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Step 1

I converted the Harry Potter novel texts from rda files to csv files using R, which is why that code isn't shown here.

I opened the first movie and book text below.

In [2]:
movie1 = pd.read_csv("philosophers_stone_movie.csv")
movie1.head(30)

,Character,Sentence
0,Dumbledore,"I should've known that you would be here, Professor McGonagall."
1,McGonagall,"Good evening, Professor Dumbledore."
2,McGonagall,"Are the rumors true, Albus?"
3,Dumbledore,"I'm afraid so, professor."
4,Dumbledore,The good and the bad.
5,McGonagall,And the boy?
6,Dumbledore,Hagrid is bringing him.
7,McGonagall,Do you think it wise to trust Hagrid with something as important as this?
8,Dumbledore,"Ah, Professor, I would trust Hagrid with my life."
9,Hagrid,"Professor Dumbledore, sir."


In [3]:
book1 = pd.read_csv("philosophers_stone_book.csv")
book1.head(30)

,Unnamed: 0,x
0,1,"THE BOY WHO LIVED Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you'd expect to be involved..."
1,2,"THE VANISHING GLASS Nearly ten years had passed since the Dursleys had woken up to find their nephew on the front step, but Privet Drive had hardly changed at all. The sun rose on the same tidy f..."
2,3,"THE LETTERS FROM NO ONE The escape of the Brazilian boa constrictor earned Harry his longest-ever punishment. By the time he was allowed out of his cupboard again, the summer holidays had started..."
3,4,"THE KEEPER OF THE KEYS BOOM. They knocked again. Dudley jerked awake. ""Where's the cannon?"" he said stupidly. There was a crash behind them and Uncle Vernon came skidding into the room. He was h..."
4,5,"DIAGON ALLEY Harry woke early the next morning. Although he could tell it was daylight, he kept his eyes shut tight. ""It was a dream"", he told himself firmly. ""I dreamed a giant called Hagrid ca..."
5,6,"THE JOURNEY FROM PLATFORM NINE AND THREE-QUARTERS Harry's last month with the Dursleys wasn't fun. True, Dudley was now so scared of Harry he wouldn't stay in the same room, while Aunt Petunia an..."
6,7,"THE SORTING HAT The door swung open at once. A tall, black-haired witch in emerald-green robes stood there. She had a very stern face and Harry's first thought was that this was not someone to cr..."
7,8,"THE POTIONS MASTER ""There, look."" ""Where?"" ""Next to the tall kid with the red hair."" ""Wearing the glasses?"" ""Did you see his face?"" ""Did you see his scar?"" Whispers followed Harry from the ..."
8,9,"THE MIDNIGHT DUEL Harry had never believed he would meet a boy he hated more than Dudley, but that was before he met Draco Malfoy. Still, first-year Gryffindors only had Potions with the Slythe..."
9,10,"HALLOWEEN Malfoy couldn't believe his eyes when he saw that Harry and Ron were still at Hogwarts the next day, looking tired but perfectly cheerful. Indeed, by the next morning Harry and Ron thou..."


## Step 2: regex

Here, I want to use regex to find every quote said by a character for the first book.

I had to manually going through the csv because some quotes are mismatched, meaning it doesn't have a beginning or end quotation mark, which messes up the data. I think it would be extremely painful to do this for each book, and this is the best dataset I could find of the text, so I am only going to compare the first book to the first movie.

In [5]:
# quotes = re.compile(r'(".*")')
# result = quotes.findall(book1.x)
# result


#matches = book1.x.str.extractall(r'(".*?")')
# matches.loc[0] \w{2,4} r'(".*?" Harry\b)'

# uncomment the two lines below only:
matches = book1.x.str.extractall(r'("[^"]+?,") (Ron|Hermione|Harry|Dumbledore|Hagrid|Malfoy|McGonagall|Petunia|Snape|Voldemort|said) (Ron|Hermione|Harry|Dumbledore|Hagrid|Malfoy|McGonagall|Petunia|Ron|Snape|Voldemort|said)')
matches = book1.x.str.extractall(r'("[^"]+?,") ([A-Z]\w+) said')
matches = book1.x.str.extractall(r'"([^"]+?)," said ((?:Mr\. |Mrs\. )?(?:[A-Z]\w+ ?)+)')
matches

#matches.loc[16]

0  \
   match                                                                                                                              
0  0                                                                                                                   I suppose so   
   1                                                                                                                        Oh, yes   
   2                                                                   You'd be stiff if you'd been sitting on a brick wall all day   
   3                                                                                                           You can't blame them   
   4                                                                                                                    I know that   
   5                                                                                                          It certainly seems so   
   6                                                                                                                  No, thank you   
   7                                                                                                                 You flatter me   
   8                                                                                                              We can only guess   
   9                                                                                                                            Yes   
   10                                                                                                   It's the best place for him   
   11                                                                                                                       Exactly   
   12                                                                                             I would trust Hagrid with my life   
   13                                                                             I'm not saying his heart isn't in the right place   
   14                                                                                                                        Hagrid   
   15                                                                                                                           Yes   
   16                                                                                                                          Well   
   17                                                                                                                          Yeah   
   18                                                                          I shall see you soon, I expect, Professor McGonagall   
1  0                                                                                                                         Nearly   
   1                                                                                                   All right, thirty-seven then   
   2                                                                                                          Thirty-nine, sweetums   
   3                                                                                                      I won't blow up the house   
   4                                                                                         I suppose we could take him to the zoo   
   5                                                                                                   I'm not going to do anything   
   6                                                                                               I had a dream about a motorcycle   
   7                                                                                                              I know they don't   
2  0                                                                                                                     No, thanks   
   1                                                                            

## Step 3

I paid each quote with a main character and created a dataframe of it. For the harry potter movies, this is already done in the original csv. First, I make sure I have a df. Then, I reset the index, renamed the columns, and now I have my pre final version of the book 1 df. Then, I added a period to the end of every quote in the df, which will be helpful for later analysis (measuring complexity needs punctuation to work). Then, I drop some of the unnecessary columns. Now, I have my final version of the book 1 df.

In [6]:
if isinstance(matches, pd.DataFrame):
    print('this is a df')

this is a df


In [7]:
matches.reset_index(inplace = True) 

In [9]:
matches.rename(columns={'match': 'match', 0: 'quotes', 1: 'character'}, inplace=True)

In [10]:
book1_prefinal = matches

In [11]:
book1_prefinal

,level_0,match,quotes,character
0,0,0,I suppose so,Mrs. Dursley
1,0,1,"Oh, yes",Mr. Dursley
2,0,2,You'd be stiff if you'd been sitting on a brick wall all day,Professor McGonagall
3,0,3,You can't blame them,Dumbledore
4,0,4,I know that,Professor McGonagall
5,0,5,It certainly seems so,Dumbledore
6,0,6,"No, thank you",Professor McGonagall
7,0,7,You flatter me,Dumbledore
8,0,8,We can only guess,Dumbledore
9,0,9,Yes,Professor McGonagall


In [12]:
book1_prefinal['quotes'] = matches['quotes'] + '.'

In [13]:
book1_final = book1_prefinal.drop(columns=['level_0', 'match'])

In [17]:
book1_final

,quotes,character
0,I suppose so.,Mrs. Dursley
1,"Oh, yes.",Mr. Dursley
2,You'd be stiff if you'd been sitting on a brick wall all day.,Professor McGonagall
3,You can't blame them.,Dumbledore
4,I know that.,Professor McGonagall
5,It certainly seems so.,Dumbledore
6,"No, thank you.",Professor McGonagall
7,You flatter me.,Dumbledore
8,We can only guess.,Dumbledore
9,Yes.,Professor McGonagall


## Actual analysis

First, I compared the first book to the first movie using tf-idf. Then I compared the two texts using sentence length (using average characters per sentence and average words per sentence). Finally, I also compared them using word complexity per character.

## tf-idf

#### for the movie

I turned the sentence column into a string in order to strip it, then did the same with the character column. I then changed the spelling of Hermione with the correct spelling. I then added a space after each sentence so when I later combined them it wouldn't lack a space after each period. I then combined the sentences per character together through aggregation. This is all normal data cleaning before analysis. 

In [19]:
movie1['Sentence'] = movie1.Sentence.astype(str)

In [20]:
movie1['Sentence'] = movie1['Sentence'].str.strip()

In [21]:
movie1['Character'] = movie1['Character'].str.strip()

In [22]:
movie1['Character'] = movie1.Character.replace('Hermoine', 'Hermione')

In [24]:
movie1['Sentence'] = movie1['Sentence'].astype(str) + ' '

In [25]:
movie_tdf = movie1.groupby('Character').agg({'Sentence' : 'sum'})
movie_tdf

,Sentence
Character,
All,AHHHHHHH! YAY!
All 3,We know about the Sorcerer's Stone!
Barkeep Tom,"Ah, Hagrid! The usual, I presume? Bless my soul. It's Harry Potter!"
Boy,"It's a world-class racing broom. Look at it! The new Nimbus 2000! It's the fastest model yet. Ravenclaw, follow me. This way. Nice going, Harry. That was wicked, Harry! Stay together!"
Class,"Good afternoon, Madam Hooch. Up! Neville! The swish and flick."
Crowd,Boooo!! Go go Gryffindor! Go go Gryffindor!
Dean,"Hey, look! Neville's got a Remembrall"
Draco,Wingardium Levio-saaa. No!
Dudley,"Wake up, cousin! We're going to the zoo! How many are there? 36?! But last year... last year I had 37! I don't care how big they are! Make it move. Move! He's boring. you won't believe what this s..."


I reset the index, chose which characters I wanted to focus on, and created a dictionary from it. 

In [27]:
movie_tdf = movie_tdf.reset_index()

In [28]:
movie_tdf_char = movie_tdf[(movie_tdf['Character'] == 'Harry') | (movie_tdf['Character'] == 'Ron') | (movie_tdf['Character'] == 'Hermione') | (movie_tdf['Character'] == 'Hagrid') | (movie_tdf['Character'] == 'Dumbledore') | (movie_tdf['Character'] == 'McGonagall') | (movie_tdf['Character'] == 'Petunia') | (movie_tdf['Character'] == 'Malfoy') | (movie_tdf['Character'] == 'Snape') | (movie_tdf['Character'] == 'Voldemort')]


In [29]:
movie_tdf_char

,Character,Sentence
9,Dumbledore,"I should've known that you would be here, Professor McGonagall. I'm afraid so, professor. The good and the bad. Hagrid is bringing him. Ah, Professor, I would trust Hagrid with my life. No problem..."
22,Hagrid,"Professor Dumbledore, sir. Professor McGonagall. No, sir. Little tyke fell asleep just as we were flying over Bristol. Try not to wake him. There you go. Sorry about that. Dry up, Dursley, you gre..."
23,Harry,"Yes, Aunt Petunia. Yes, Uncle Vernon. He's asleep! Sorry about him. He doesn't understand what it's like, lying there day after day... ...watching people press their ugly faces in on you. Can you ..."
24,Hermione,"Has anyone seen a toad? A boy named Neville's lost one Oh, are you doing magic? Let's see, then. Are you sure that's a real spell? Well, it's not very good, is it? Of course, I've only tried a few..."
28,Malfoy,"It's true then, what they're saying on the train. Harry Potter has come to Hogwarts. This is Crabbe, and Goyle. And I'm Malfoy. Draco Malfoy. Think my name's funny, do you? I've no need to ask you..."
31,McGonagall,"Good evening, Professor Dumbledore. Are the rumors true, Albus? And the boy? Do you think it wise to trust Hagrid with something as important as this? I've watched them all day. They're the worst ..."
38,Petunia,"Up. Get up! Now! Here he comes, the birthday boy. Why don't you just cook the breakfast, and try not to burn anything. I want everything to be perfect for my Dudley's special day! Aren't they wond..."
40,Ron,"Excuse me. Do you mind? Everywhere else is full. I'm Ron, by the way. Ron Weasley. So it's true! I mean, do you really have the...? The scar? Wicked! No, thanks. I'm all set. They mean every flavo..."
45,Snape,"There will be no foolish wand-waving or silly incantations in this class. As such, I don't expect many of you to appreciate the subtle science and exact art that is potion-making. However, for tho..."
51,Voldemort,Use the boy He lies. Let me speak to him. I have strength enough for this. Harry Potter. We meet again. Yes. You see what I have become? See what I must do to survive? Live off another. A mere par...


In [30]:
movie_dict = movie_tdf_char.set_index('Character').T.to_dict('list')

In [31]:
movie_dict

{'Dumbledore': ["I should've known that you would be here, Professor McGonagall. I'm afraid so, professor. The good and the bad. Hagrid is bringing him. Ah, Professor, I would trust Hagrid with my life. No problems, I trust, Hagrid? Albus, do you really think it's safe, leaving him with these people? The only family he has. Exactly. He's far better off growing up away from all of that. Until he's ready. There, there, Hagrid. It's not really goodbye, after all. Good luck...Harry Potter. I have a few start-of-term notices I wish to announce. The first years, please note that the Dark Forest is strictly forbidden to all students. Also, our caretaker, Mr. Filch, has asked me to remind you that the third-floor corridor on the right hand side is out of bounds to everyone who does not wish to die a most painful death. Thank you. Let the feast begin. SILLLLLEEENNNNCEEEEE! Everyone will please, not panic. Now, Prefects will lead their houses back to the dormitories. Teachers will follow me to t

I joined all the quotes per character together after creating a list of them. I then added my stop words and tokenized the words using the code below. From there, I added my list of speakers, tested it worked, and ran a vectorizer. 

In [32]:
quotes = list(movie_dict.values())

In [33]:
quotes_str = ["".join(x) for x in quotes]

In [34]:
stop_words = stopwords.words('english') + list(punctuation)

clean_quotes = []

for x in quotes_str:
    x = word_tokenize(x)
    x = [w.lower() for w in x]
    # remove digits useing .isdigit()
    x = [w for w in x if not w.isdigit()]
    # remove stop words here
    x = [w for w in x if w not in stop_words]
    x = " ".join(x)
    clean_quotes.append(x)

In [35]:
speakers = list(movie_dict.keys())

Dumbledore, Hagrid, Harry, Hermione, Malfoy, McGonagall, Petunia, Ron, Snape, Voldemort = [y for y in clean_quotes]

In [36]:
speakers

['Dumbledore',
 'Hagrid',
 'Harry',
 'Hermione',
 'Malfoy',
 'McGonagall',
 'Petunia',
 'Ron',
 'Snape',
 'Voldemort']

In [38]:
# replace x and y with your chosen ngram range.
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
# list your documents as a list where instructed. Use the speaker names as variables, not as literals.
vectors = vectorizer.fit_transform([Dumbledore, Hagrid, Harry, Hermione, Malfoy, McGonagall, Petunia, Ron, Snape, Voldemort])
# these are all the ngrams in the corpus you created. take a peak!
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
# creating a dataframe with the  feature names as columns and your previously created speakers list as rows. make sure to update the index list if you removed some speakers.
df = pd.DataFrame(denselist, columns=feature_names, index=speakers)
# transpose dataframe so the names are the columns and rows are the features - easier to analyze.
df_t = df.T

The results are below! Hagrid, Harry, and Hermione all say Voldemort more than they should. Harry says other people's names more than anything else, and Hermione, Ron, Draco, Hagrid, and Dumbledore both say Harry's name A LOT. Dumbledore also loves to award points. McGonagall references Hermione more than Harry. The first cell below shows every word and how it compares per character (how many times each one said it). The next cells show the top words per character listed, and the last cell shows how often the characters say Voldemort.

In [39]:
df.T

,Dumbledore,Hagrid,Harry,Hermione,Malfoy,McGonagall,Petunia,Ron,Snape,Voldemort
15th,0.000000,0.000000,0.026040,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3rd,0.000000,0.000000,0.000000,0.084963,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
665th,0.000000,0.000000,0.000000,0.042482,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
able,0.046724,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
abnormal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.101085,0.000000,0.0,0.000000
absolutely,0.000000,0.028130,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
accepted,0.000000,0.000000,0.026040,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
accident,0.000000,0.000000,0.000000,0.042482,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
according,0.000000,0.000000,0.022136,0.036113,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
account,0.046724,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [40]:
df_t.nlargest(5,'Harry')

,Dumbledore,Hagrid,Harry,Hermione,Malfoy,McGonagall,Petunia,Ron,Snape,Voldemort
hagrid,0.110984,0.016704,0.324722,0.000000,0.00000,0.037943,0.000000,0.045714,0.0,0.0
ron,0.000000,0.018601,0.223837,0.112360,0.00000,0.000000,0.000000,0.076356,0.0,0.0
know,0.025083,0.120810,0.209683,0.114027,0.00000,0.102904,0.000000,0.165308,0.0,0.0
snape,0.000000,0.074403,0.172182,0.056180,0.00000,0.000000,0.000000,0.050904,0.0,0.0
one,0.041483,0.124875,0.150272,0.075433,0.05218,0.056729,0.044873,0.119609,0.0,0.0


In [41]:
#they’re and they’ve
df_t.nlargest(5,'Hermione')

,Dumbledore,Hagrid,Harry,Hermione,Malfoy,McGonagall,Petunia,Ron,Snape,Voldemort
re,0.020742,0.149850,0.080916,0.207439,0.052180,0.085093,0.134619,0.153783,0.0,0.000000
ve,0.022776,0.013712,0.114240,0.207081,0.057299,0.031147,0.000000,0.150105,0.0,0.000000
harry,0.165932,0.387111,0.057797,0.188581,0.052180,0.056729,0.000000,0.290478,0.0,0.185816
read,0.000000,0.000000,0.022136,0.180567,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
going,0.000000,0.083522,0.077315,0.151359,0.000000,0.000000,0.240106,0.022857,0.0,0.000000


In [93]:
df_t.nlargest(5,'Ron')

,Dumbledore,Hagrid,Harry,Hermione,Malfoy,McGonagall,Petunia,Ron,Snape,Voldemort
harry,0.165932,0.387111,0.057797,0.188581,0.05218,0.056729,0.000000,0.290478,0.0,0.185816
got,0.000000,0.100227,0.123704,0.075680,0.00000,0.000000,0.180079,0.228572,0.0,0.000000
go,0.020742,0.174824,0.127153,0.037716,0.10436,0.028364,0.224364,0.170870,0.0,0.000000
know,0.025083,0.120810,0.209683,0.114027,0.00000,0.102904,0.000000,0.165308,0.0,0.000000
re,0.020742,0.149850,0.080916,0.207439,0.05218,0.085093,0.134619,0.153783,0.0,0.000000


In [94]:
df_t.nlargest(5,'Dumbledore')

,Dumbledore,Hagrid,Harry,Hermione,Malfoy,McGonagall,Petunia,Ron,Snape,Voldemort
points,0.397200,0.000000,0.000000,0.000000,0.00000,0.325907,0.0,0.000000,0.0,0.000000
ah,0.280346,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000
harry,0.165932,0.387111,0.057797,0.188581,0.05218,0.056729,0.0,0.290478,0.0,0.185816
award,0.140173,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000
third,0.140173,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000


In [99]:
df_t.nlargest(5,'McGonagall')

,Dumbledore,Hagrid,Harry,Hermione,Malfoy,McGonagall,Petunia,Ron,Snape,Voldemort
points,0.397200,0.000000,0.000000,0.000000,0.000000,0.325907,0.0,0.0,0.0,0.0
wood,0.000000,0.000000,0.000000,0.000000,0.000000,0.191690,0.0,0.0,0.0,0.0
granger,0.034750,0.000000,0.000000,0.031595,0.000000,0.142565,0.0,0.0,0.0,0.0
professor,0.125415,0.105709,0.041937,0.045611,0.063102,0.137206,0.0,0.0,0.0,0.0
awarded,0.000000,0.000000,0.000000,0.000000,0.000000,0.127793,0.0,0.0,0.0,0.0


In [42]:
df['voldemort']

Dumbledore    0.000000
Hagrid        0.037201
Harry         0.120528
Hermione      0.028090
Malfoy        0.000000
McGonagall    0.000000
Petunia       0.000000
Ron           0.000000
Snape         1.000000
Voldemort     0.000000
Name: voldemort, dtype: float64

#### for the book

Exactly the same as the movie, I turned the quote column into a string in order to strip it, then did the same with the character column. I then changed Aunt Petunia and Professor McGonagall to be one word, so it could be run through the vectorizer. I then stipped both columns and added a space after each sentence so when I later combined them it wouldn't lack a space after each period. I then combined the sentences per character together through aggregation. This is all normal data cleaning before analysis.

In [44]:
book1_final['quotes'] = book1_final.quotes.astype(str)

In [45]:
book1_final['character'] = book1_final.character.replace('Aunt Petunia', 'Aunt_Petunia')

In [46]:
book1_final['character'] = book1_final.character.replace('Professor McGonagall', 'Professor_McGonagall')

In [47]:
book1_final['quotes'] = book1_final['quotes'].str.strip()

In [48]:
book1_final['character'] = book1_final['character'].str.strip()

In [50]:
book1_final['quotes'] = book1_final['quotes'].astype(str) + ' '

In [51]:
book1_tdf = book1_final.groupby('character').agg({'quotes' : 'sum'})
book1_tdf

,quotes
character,
Aunt Petunia,I suppose we could take him to the zoo.
Aunt_Petunia,"Thirty-nine, sweetums."
Chaser Angelina Johnson,And women.
Draco Malfoy,I do feel so sorry.
Dudley,"All right, thirty-seven then."
Dumbledore,"You can't blame them. It certainly seems so. You flatter me. We can only guess. It's the best place for him. Exactly. I would trust Hagrid with my life. Hagrid. Yes. Well. I shall see you soon, I ..."
Filch,"Follow me. I shouldn't be too friendly to them, Hagrid. I'll be back at dawn."
Firenze,"That is because it is a monstrous thing, to slay a unicorn. Good luck, Harry Potter."
Fred,"I tell you, we're going to win that Quidditch cup for sure this year. Harry's is better than ours, though."


I reset the index, chose which characters I wanted to focus on, and created a dictionary from it. 

In [52]:
book1_tdf = book1_tdf.reset_index()

In [53]:
book_tdf_char = book1_tdf[(book1_tdf['character'] == 'Harry') | (book1_tdf['character'] == 'Ron') | (book1_tdf['character'] == 'Hermione') | (book1_tdf['character'] == 'Hagrid') | (book1_tdf['character'] == 'Dumbledore') | (book1_tdf['character'] == 'Professor_McGonagall') | (book1_tdf['character'] == 'Aunt_Petunia') | (book1_tdf['character'] == 'Malfoy') | (book1_tdf['character'] == 'Snape') | (book1_tdf['character'] == 'Voldemort')]


In [54]:
book_tdf_char

,character,quotes
1,Aunt_Petunia,"Thirty-nine, sweetums."
5,Dumbledore,"You can't blame them. It certainly seems so. You flatter me. We can only guess. It's the best place for him. Exactly. I would trust Hagrid with my life. Hagrid. Yes. Well. I shall see you soon, I ..."
12,Hagrid,"Yeah. Ah, go boil yet heads, both of yeh. A wizard, o' course. Gallopin' Gorgons, that reminds me. A Muggle. That's better. It's gettin' late and we've got lots ter do tomorrow. Give him five Knut..."
13,Harry,"Nearly. I won't blow up the house. I'm not going to do anything. I had a dream about a motorcycle. I know they don't. No, thanks. want to read it. It was not a mistake. Er -- no. Of course not. Ha..."
14,Hermione,"Goodness, didn't you know, I'd have found out everything I could if it was me. All right -- I only came in here because people outside are behaving very childishly, racing up and down the corridor..."
16,Malfoy,"I'd take you on anytime on my own. Potter's been sent a broomstick, Professor. I want Fang."
27,Professor_McGonagall,"You'd be stiff if you'd been sitting on a brick wall all day. Yes. Welcome to Hogwarts. I shall return when we are ready for you. Follow me, you two. Miss Granger, five points will be taken from G..."
29,Ron,"Shut up. Shut up. Oh -well, I thought it might be one of Fred and George's jokes. Wow. Er -- Yes, I think so. I heard you went to live with Muggles. Five. You don't want this, it's all dry. No. We..."
33,Snape,"Library books are not to be taken outside the school. Be that as it may, fighting is against Hogwarts rules, Hagrid. Oh, I thought we'd keep this private. You don't want me as your enemy, Quirrell..."


In [55]:
book_dict = book_tdf_char.set_index('character').T.to_dict('list')

In [56]:
book_dict

{'Aunt_Petunia': ['Thirty-nine, sweetums. '],
 'Dumbledore': ["You can't blame them. It certainly seems so. You flatter me. We can only guess. It's the best place for him. Exactly. I would trust Hagrid with my life. Hagrid. Yes. Well. I shall see you soon, I expect, Professor McGonagall. Everyone pick their favorite tune. Strange how nearsighted being invisible can make you. So. I don't need a cloak to become invisible. Yes and no. One can never have enough socks. Well done. Good afternoon, Harry. Calm yourself, dear boy, you are a little behind the times. Tokens from your friends and admirers. Yes, Yes, well done, Slytherin. Ahem. There are all kinds of courage. "],
 'Hagrid': ["Yeah. Ah, go boil yet heads, both of yeh. A wizard, o' course. Gallopin' Gorgons, that reminds me. A Muggle. That's better. It's gettin' late and we've got lots ter do tomorrow. Give him five Knuts. Don't worry about that. Flew. Seems a shame ter row, though. Spells -- enchantments. 'Course. Well, so they say.

I joined all the quotes per character together after creating a list of them. I then added my stop words and tokenized the words using the code below. From there, I added my list of speakers, tested it worked, and ran a vectorizer.

In [57]:
quotes1 = list(book_dict.values())

In [58]:
quotes_str1 = ["".join(x) for x in quotes]

In [59]:
stop_words = stopwords.words('english') + list(punctuation)

clean_quotes = []

for x in quotes_str1:
    x = word_tokenize(x)
    x = [w.lower() for w in x]
    # remove digits useing .isdigit()
    x = [w for w in x if not w.isdigit()]
    # remove stop words here
    x = [w for w in x if w not in stop_words]
    x = " ".join(x)
    clean_quotes.append(x)

In [60]:
speakers = list(book_dict.keys())

Dumbledore, Hagrid, Harry, Hermione, Malfoy, Professor_McGonagall, Aunt_Petunia, Ron, Snape, Voldemort = [y for y in clean_quotes]










In [61]:
speakers

['Aunt_Petunia',
 'Dumbledore',
 'Hagrid',
 'Harry',
 'Hermione',
 'Malfoy',
 'Professor_McGonagall',
 'Ron',
 'Snape']

In [63]:
# replace x and y with your chosen ngram range.
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
# list your documents as a list where instructed. Use the speaker names as variables, not as literals.
vectors = vectorizer.fit_transform([Dumbledore, Hagrid, Harry, Hermione, Malfoy, Professor_McGonagall, Aunt_Petunia, Ron, Snape])
# these are all the ngrams in the corpus you created. take a peak!
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
# creating a dataframe with the  feature names as columns and your previously created speakers list as rows. make sure to update the index list if you removed some speakers.
df1 = pd.DataFrame(denselist, columns=feature_names, index=speakers)
# transpose dataframe so the names are the columns and rows are the features - easier to analyze.
df1_t = df1.T

The results are below! In the first book, Dumbledore, Hagrid, and Harry all say Voldemort more than they should (compared to the movie, this is very different). Every character says other people's names a lot less than in the movie as well. The first cell below shows every word and how it compares per character (how many times each one said it). The next cells show the top words per character listed, and the next one after those show how often the characters say Voldemort. The last type of analysis done on this data is seeing the difference in each word said in the book and movie for Harry Potter himself.

In [64]:
df1.T

,Aunt_Petunia,Dumbledore,Hagrid,Harry,Hermione,Malfoy,Professor_McGonagall,Ron,Snape
15th,0.000000,0.000000,0.026254,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3rd,0.000000,0.000000,0.000000,0.085307,0.000000,0.000000,0.000000,0.000000,0.0
665th,0.000000,0.000000,0.000000,0.042653,0.000000,0.000000,0.000000,0.000000,0.0
able,0.046707,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
abnormal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.101270,0.000000,0.0
absolutely,0.000000,0.028186,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
accepted,0.000000,0.000000,0.026254,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
accident,0.000000,0.000000,0.000000,0.042653,0.000000,0.000000,0.000000,0.000000,0.0
according,0.000000,0.000000,0.022175,0.036026,0.000000,0.000000,0.000000,0.000000,0.0
account,0.046707,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [104]:
df1_t.nlargest(5,'Professor_McGonagall')

,Aunt_Petunia,Dumbledore,Hagrid,Harry,Hermione,Malfoy,Professor_McGonagall,Ron,Snape
get,0.021893,0.026424,0.110756,0.039986,0.055355,0.000000,0.237346,0.018138,0.0
day,0.000000,0.016319,0.091204,0.000000,0.000000,0.037068,0.234535,0.022404,0.0
going,0.000000,0.081597,0.076003,0.148174,0.000000,0.000000,0.234535,0.022404,0.0
go,0.019785,0.167157,0.122333,0.036136,0.100050,0.027120,0.214490,0.163914,0.0
darling,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.202540,0.000000,0.0


In [105]:
df1_t.nlargest(5,'Dumbledore')

,Aunt_Petunia,Dumbledore,Hagrid,Harry,Hermione,Malfoy,Professor_McGonagall,Ron,Snape
harry,0.175146,0.409573,0.061531,0.199932,0.055355,0.060019,0.000000,0.308346,0.0
ooh,0.000000,0.197305,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
well,0.043787,0.184968,0.036919,0.039986,0.000000,0.030010,0.047469,0.072552,0.0
go,0.019785,0.167157,0.122333,0.036136,0.100050,0.027120,0.214490,0.163914,0.0
oh,0.000000,0.161198,0.054599,0.066528,0.000000,0.066572,0.052651,0.100591,0.0


In [67]:
df1['voldemort']

Aunt_Petunia            0.000000
Dumbledore              0.041398
Hagrid                  0.134961
Harry                   0.031323
Hermione                0.000000
Malfoy                  0.000000
Professor_McGonagall    0.000000
Ron                     0.000000
Snape                   0.000000
Name: voldemort, dtype: float64

In [65]:
df1.T['Harry'] - df.T['Harry']

15th                   -0.026040
3rd                     0.085307
665th                   0.042653
able                    0.000000
abnormal                0.000000
absolutely              0.000000
accepted               -0.026040
accident                0.042653
according               0.013889
account                 0.000000
acknowledging          -0.026040
across                  0.000000
act                     0.000000
actions                 0.000000
actually                0.000000
admirers               -0.022136
affairs                 0.000000
afraid                  0.000000
afternoon               0.000000
ago                     0.036026
agreed                  0.000000
ah                      0.000000
ahh                    -0.022136
ahhh                    0.031323
ahhhh                   0.000000
ahhhhhhhhh             -0.022136
ai                      0.000000
alas                    0.000000
albus                   0.000000
alchemist               0.042653
alchemy   

## sentence length

#### for the movie

The code below calculates the average number of characters per sentence. I then chose which characters from my df I wanted to calculate it for. Snape uses the most characters and Ron uses the least.

In [68]:
df = (movie1.groupby('Character')['Sentence']
                            .apply(lambda x: np.mean(x.str.len()))
                            .reset_index(name='avg_characters_per_sentence'))

In [69]:
df1 = df[(df['Character'] == 'Harry') | (df['Character'] == 'Ron') | (df['Character'] == 'Hermione') | (df['Character'] == 'Hagrid') | (df['Character'] == 'Dumbledore') | (df['Character'] == 'McGonagall') | (df['Character'] == 'Petunia') | (df['Character'] == 'Malfoy') | (df['Character'] == 'Snape') | (df['Character'] == 'Voldemort')]

In [70]:
df1

,Character,avg_characters_per_sentence
9,Dumbledore,53.853333
22,Hagrid,36.548837
23,Harry,27.821212
24,Hermione,31.294798
28,Malfoy,30.666667
31,McGonagall,45.437500
38,Petunia,31.844444
40,Ron,27.000000
45,Snape,58.090909
51,Voldemort,32.153846


The code below calculates the average number of words per sentence for each character. Snape says the most words per sentence on average and Malfoy says the least.

In [72]:
def get_average_words(sentence):
    dum1 = sentence.split(".")
    return sum(len(x.split()) for x in dum1) / len(dum1)
    
movie_tdf_char['avg words per sentence'] = movie_tdf_char.Sentence.apply(get_average_words)
movie_tdf_char

/Users/lindseyfeingold/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Character,Sentence,avg words per sentence
9,Dumbledore,"I should've known that you would be here, Professor McGonagall. I'm afraid so, professor. The good and the bad. Hagrid is bringing him. Ah, Professor, I would trust Hagrid with my life. No problem...",8.303371
22,Hagrid,"Professor Dumbledore, sir. Professor McGonagall. No, sir. Little tyke fell asleep just as we were flying over Bristol. Try not to wake him. There you go. Sorry about that. Dry up, Dursley, you gre...",6.642202
23,Harry,"Yes, Aunt Petunia. Yes, Uncle Vernon. He's asleep! Sorry about him. He doesn't understand what it's like, lying there day after day... ...watching people press their ugly faces in on you. Can you ...",6.503817
24,Hermione,"Has anyone seen a toad? A boy named Neville's lost one Oh, are you doing magic? Let's see, then. Are you sure that's a real spell? Well, it's not very good, is it? Of course, I've only tried a few...",7.111111
28,Malfoy,"It's true then, what they're saying on the train. Harry Potter has come to Hogwarts. This is Crabbe, and Goyle. And I'm Malfoy. Draco Malfoy. Think my name's funny, do you? I've no need to ask you...",6.375000
31,McGonagall,"Good evening, Professor Dumbledore. Are the rumors true, Albus? And the boy? Do you think it wise to trust Hagrid with something as important as this? I've watched them all day. They're the worst ...",7.878788
38,Petunia,"Up. Get up! Now! Here he comes, the birthday boy. Why don't you just cook the breakfast, and try not to burn anything. I want everything to be perfect for my Dudley's special day! Aren't they wond...",8.937500
40,Ron,"Excuse me. Do you mind? Everywhere else is full. I'm Ron, by the way. Ron Weasley. So it's true! I mean, do you really have the...? The scar? Wicked! No, thanks. I'm all set. They mean every flavo...",6.886667
45,Snape,"There will be no foolish wand-waving or silly incantations in this class. As such, I don't expect many of you to appreciate the subtle science and exact art that is potion-making. However, for tho...",10.147059
51,Voldemort,Use the boy He lies. Let me speak to him. I have strength enough for this. Harry Potter. We meet again. Yes. You see what I have become? See what I must do to survive? Live off another. A mere par...,7.950000


In [73]:
# hag1 = movie_tdf_char["Sentence"].loc[22].split('.')
# avg_len = sum(len(x.split()) for x in hag1) / len(hag1)
# print(f'Hagrid has an average of {avg_len} words per sentence')

#### for the book

The code below calculates the average number of characters per sentence. Harry says the least while Snape, similarly to the movie, says the most.

In [112]:
df2 = (book1_final.groupby('character')['quotes']
                            .apply(lambda x: np.mean(x.str.len()))
                            .reset_index(name='avg_characters_per_sentence'))

In [117]:
df3 = df2[(df2['character'] == 'Harry') | (df2['character'] == 'Ron') | (df2['character'] == 'Hermione') | (df2['character'] == 'Hagrid') | (df2['character'] == 'Dumbledore') | (df2['character'] == 'Professor_McGonagall') | (df2['character'] == 'Aunt_Petunia') | (df2['character'] == 'Malfoy') | (df2['character'] == 'Snape') | (df2['character'] == 'Voldemort')]

In [118]:
df3

,character,avg_characters_per_sentence
1,Aunt_Petunia,23.000000
5,Dumbledore,25.375000
12,Hagrid,27.178082
13,Harry,20.082474
14,Hermione,34.103448
16,Malfoy,30.666667
27,Professor_McGonagall,33.777778
29,Ron,30.250000
33,Snape,45.400000


The code below calculates the average number of words per sentence for each character. Snape says the most words per sentence on average, like the movie, and Aunt Petunia says the least.

In [76]:
def get_average_words(quotes):
    dum2 = quotes.split(".")
    return sum(len(x.split()) for x in dum2) / len(dum2)
    
book_tdf_char['avg words per sentence'] = book_tdf_char.quotes.apply(get_average_words)
book_tdf_char

/Users/lindseyfeingold/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,character,quotes,avg words per sentence
1,Aunt_Petunia,"Thirty-nine, sweetums.",1.000000
5,Dumbledore,"You can't blame them. It certainly seems so. You flatter me. We can only guess. It's the best place for him. Exactly. I would trust Hagrid with my life. Hagrid. Yes. Well. I shall see you soon, I ...",4.240000
12,Hagrid,"Yeah. Ah, go boil yet heads, both of yeh. A wizard, o' course. Gallopin' Gorgons, that reminds me. A Muggle. That's better. It's gettin' late and we've got lots ter do tomorrow. Give him five Knut...",4.878378
13,Harry,"Nearly. I won't blow up the house. I'm not going to do anything. I had a dream about a motorcycle. I know they don't. No, thanks. want to read it. It was not a mistake. Er -- no. Of course not. Ha...",3.693878
14,Hermione,"Goodness, didn't you know, I'd have found out everything I could if it was me. All right -- I only came in here because people outside are behaving very childishly, racing up and down the corridor...",5.575758
16,Malfoy,"I'd take you on anytime on my own. Potter's been sent a broomstick, Professor. I want Fang.",4.250000
27,Professor_McGonagall,"You'd be stiff if you'd been sitting on a brick wall all day. Yes. Welcome to Hogwarts. I shall return when we are ready for you. Follow me, you two. Miss Granger, five points will be taken from G...",5.800000
29,Ron,"Shut up. Shut up. Oh -well, I thought it might be one of Fred and George's jokes. Wow. Er -- Yes, I think so. I heard you went to live with Muggles. Five. You don't want this, it's all dry. No. We...",5.532468
33,Snape,"Library books are not to be taken outside the school. Be that as it may, fighting is against Hogwarts rules, Hagrid. Oh, I thought we'd keep this private. You don't want me as your enemy, Quirrell...",7.000000


## Complexity

#### for the movie

To calculate complexity, I first did a little data cleaning by dropping the average words per sentence column, since we no longer need that data. Then I used a textatistic library on each character listed below to calculate character count, word count, sentence count, syllable count, and other scores including the Flesch score (the lower the score, the more difficult the text is to read) and the Dalechall score (it measures a text against a number of words considered familiar to fourth-graders, and the more unfamiliar words used, the higher the reading level will be). I found that Dumbledore's sentences are the most complex, according to the Flesch score, and that Hermione's sentences are the most complex according to the Dalechall score.

In [78]:
movie_tdf_char

,Character,Sentence,avg words per sentence
9,Dumbledore,"I should've known that you would be here, Professor McGonagall. I'm afraid so, professor. The good and the bad. Hagrid is bringing him. Ah, Professor, I would trust Hagrid with my life. No problem...",8.303371
22,Hagrid,"Professor Dumbledore, sir. Professor McGonagall. No, sir. Little tyke fell asleep just as we were flying over Bristol. Try not to wake him. There you go. Sorry about that. Dry up, Dursley, you gre...",6.642202
23,Harry,"Yes, Aunt Petunia. Yes, Uncle Vernon. He's asleep! Sorry about him. He doesn't understand what it's like, lying there day after day... ...watching people press their ugly faces in on you. Can you ...",6.503817
24,Hermione,"Has anyone seen a toad? A boy named Neville's lost one Oh, are you doing magic? Let's see, then. Are you sure that's a real spell? Well, it's not very good, is it? Of course, I've only tried a few...",7.111111
28,Malfoy,"It's true then, what they're saying on the train. Harry Potter has come to Hogwarts. This is Crabbe, and Goyle. And I'm Malfoy. Draco Malfoy. Think my name's funny, do you? I've no need to ask you...",6.375000
31,McGonagall,"Good evening, Professor Dumbledore. Are the rumors true, Albus? And the boy? Do you think it wise to trust Hagrid with something as important as this? I've watched them all day. They're the worst ...",7.878788
38,Petunia,"Up. Get up! Now! Here he comes, the birthday boy. Why don't you just cook the breakfast, and try not to burn anything. I want everything to be perfect for my Dudley's special day! Aren't they wond...",8.937500
40,Ron,"Excuse me. Do you mind? Everywhere else is full. I'm Ron, by the way. Ron Weasley. So it's true! I mean, do you really have the...? The scar? Wicked! No, thanks. I'm all set. They mean every flavo...",6.886667
45,Snape,"There will be no foolish wand-waving or silly incantations in this class. As such, I don't expect many of you to appreciate the subtle science and exact art that is potion-making. However, for tho...",10.147059
51,Voldemort,Use the boy He lies. Let me speak to him. I have strength enough for this. Harry Potter. We meet again. Yes. You see what I have become? See what I must do to survive? Live off another. A mere par...,7.950000


In [80]:
movie_tdf_char = movie_tdf_char.drop(columns=['avg words per sentence'])

In [81]:
dumbledore = Textatistic(movie_tdf_char["Sentence"].loc[9])
dumbledore_values = dumbledore.dict()
df = pd.DataFrame(list(dumbledore_values.items()), columns= ['metric', 'value'])
print('dumbledore', (tabulate(df, headers='keys', tablefmt='psql')))

dumbledore +----+---------------------+------------+
|    | metric              |      value |
|----+---------------------+------------|
|  0 | char_count          | 3303       |
|  1 | word_count          |  739       |
|  2 | sent_count          |   94       |
|  3 | sybl_count          |  899       |
|  4 | notdalechall_count  |  102       |
|  5 | polysyblword_count  |   30       |
|  6 | flesch_score        |   95.9387  |
|  7 | fleschkincaid_score |    1.83087 |
|  8 | gunningfog_score    |    4.7685  |
|  9 | smog_score          |    6.35642 |
| 10 | dalechall_score     |    6.20585 |
+----+---------------------+------------+


In [82]:
hagrid = Textatistic(movie_tdf_char["Sentence"].loc[22])
hagrid_values = hagrid.dict()
df = pd.DataFrame(list(hagrid_values.items()), columns= ['metric', 'value'])
print('hagrid', (tabulate(df, headers='keys', tablefmt='psql')))

hagrid +----+---------------------+-------------+
|    | metric              |       value |
|----+---------------------+-------------|
|  0 | char_count          | 6411        |
|  1 | word_count          | 1450        |
|  2 | sent_count          |  273        |
|  3 | sybl_count          | 1694        |
|  4 | notdalechall_count  |  266        |
|  5 | polysyblword_count  |   41        |
|  6 | flesch_score        |  102.608    |
|  7 | fleschkincaid_score |    0.267084 |
|  8 | gunningfog_score    |    3.25558  |
|  9 | smog_score          |    5.34299  |
| 10 | dalechall_score     |    6.79659  |
+----+---------------------+-------------+


In [83]:
harry = Textatistic(movie_tdf_char["Sentence"].loc[23])
harry_values = harry.dict()
df = pd.DataFrame(list(harry_values.items()), columns= ['metric', 'value'])
print('harry', (tabulate(df, headers='keys', tablefmt='psql')))

harry +----+---------------------+-------------+
|    | metric              |       value |
|----+---------------------+-------------|
|  0 | char_count          | 7495        |
|  1 | word_count          | 1693        |
|  2 | sent_count          |  442        |
|  3 | sybl_count          | 1982        |
|  4 | notdalechall_count  |  282        |
|  5 | polysyblword_count  |   28        |
|  6 | flesch_score        |  103.906    |
|  7 | fleschkincaid_score |   -0.281882 |
|  8 | gunningfog_score    |    2.19367  |
|  9 | smog_score          |    4.56695  |
| 10 | dalechall_score     |    6.4566   |
+----+---------------------+-------------+


In [84]:
hermione = Textatistic(movie_tdf_char["Sentence"].loc[24])
hermione_values = hermione.dict()
df = pd.DataFrame(list(hermione_values.items()), columns= ['metric', 'value'])
print('hermione', (tabulate(df, headers='keys', tablefmt='psql')))

hermione +----+---------------------+-------------+
|    | metric              |       value |
|----+---------------------+-------------|
|  0 | char_count          | 4461        |
|  1 | word_count          |  956        |
|  2 | sent_count          |  215        |
|  3 | sybl_count          | 1171        |
|  4 | notdalechall_count  |  204        |
|  5 | polysyblword_count  |   36        |
|  6 | flesch_score        |   98.6956   |
|  7 | fleschkincaid_score |    0.597905 |
|  8 | gunningfog_score    |    3.28488  |
|  9 | smog_score          |    5.46674  |
| 10 | dalechall_score     |    7.22646  |
+----+---------------------+-------------+


In [85]:
ron = Textatistic(movie_tdf_char["Sentence"].loc[40])
ron_values = ron.dict()
df = pd.DataFrame(list(ron_values.items()), columns= ['metric', 'value'])
print('ron', (tabulate(df, headers='keys', tablefmt='psql')))

ron +----+---------------------+-------------+
|    | metric              |       value |
|----+---------------------+-------------|
|  0 | char_count          | 4646        |
|  1 | word_count          | 1032        |
|  2 | sent_count          |  261        |
|  3 | sybl_count          | 1207        |
|  4 | notdalechall_count  |  216        |
|  5 | polysyblword_count  |   26        |
|  6 | flesch_score        |  103.876    |
|  7 | fleschkincaid_score |   -0.246962 |
|  8 | gunningfog_score    |    2.58936  |
|  9 | smog_score          |    4.93216  |
| 10 | dalechall_score     |    7.1375   |
+----+---------------------+-------------+


#### for the book

Exactly the same as the movie, to calculate complexity, I first did a little data cleaning by dropping the average words per sentence column since we no longer need that data. Then I used a textatistic library on each character listed below to calculate character count, word count, sentence count, syllable count, and other scores including the Flesch score (the lower the score, the more difficult the text is to read) and the Dalechall score (it measures a text against a number of words considered familiar to fourth-graders, and the more unfamiliar words used, the higher the reading level will be). I found that Dumbledore's sentences are still the most complex (same as the movie), according to the Flesch score, and that shockingly Hagrid's sentences are the most complex according to the Dalechall score, which listed Hermione as the most complex in the movie. However, that could be because Hagrid says a lot of words in an accent, which J.K. Rowling emphasizes in how she spells words he says. A list of words a fourth-grader would know definitely wouldn't include "yeh" or "gettin'", it would include "your" and "getting" since that is the correct spelling.

In [86]:
book_tdf_char = book_tdf_char.drop(columns=['avg words per sentence'])
book_tdf_char

,character,quotes
1,Aunt_Petunia,"Thirty-nine, sweetums."
5,Dumbledore,"You can't blame them. It certainly seems so. You flatter me. We can only guess. It's the best place for him. Exactly. I would trust Hagrid with my life. Hagrid. Yes. Well. I shall see you soon, I ..."
12,Hagrid,"Yeah. Ah, go boil yet heads, both of yeh. A wizard, o' course. Gallopin' Gorgons, that reminds me. A Muggle. That's better. It's gettin' late and we've got lots ter do tomorrow. Give him five Knut..."
13,Harry,"Nearly. I won't blow up the house. I'm not going to do anything. I had a dream about a motorcycle. I know they don't. No, thanks. want to read it. It was not a mistake. Er -- no. Of course not. Ha..."
14,Hermione,"Goodness, didn't you know, I'd have found out everything I could if it was me. All right -- I only came in here because people outside are behaving very childishly, racing up and down the corridor..."
16,Malfoy,"I'd take you on anytime on my own. Potter's been sent a broomstick, Professor. I want Fang."
27,Professor_McGonagall,"You'd be stiff if you'd been sitting on a brick wall all day. Yes. Welcome to Hogwarts. I shall return when we are ready for you. Follow me, you two. Miss Granger, five points will be taken from G..."
29,Ron,"Shut up. Shut up. Oh -well, I thought it might be one of Fred and George's jokes. Wow. Er -- Yes, I think so. I heard you went to live with Muggles. Five. You don't want this, it's all dry. No. We..."
33,Snape,"Library books are not to be taken outside the school. Be that as it may, fighting is against Hogwarts rules, Hagrid. Oh, I thought we'd keep this private. You don't want me as your enemy, Quirrell..."


In [87]:
dumbledore1 = Textatistic(book_tdf_char["quotes"].loc[5])
dumbledore_values1 = dumbledore1.dict()
df = pd.DataFrame(list(dumbledore_values1.items()), columns= ['metric', 'value'])
print('dumbledore', (tabulate(df, headers='keys', tablefmt='psql')))

dumbledore +----+---------------------+-----------+
|    | metric              |     value |
|----+---------------------+-----------|
|  0 | char_count          | 503       |
|  1 | word_count          | 106       |
|  2 | sent_count          |  24       |
|  3 | sybl_count          | 135       |
|  4 | notdalechall_count  |  17       |
|  5 | polysyblword_count  |   6       |
|  6 | flesch_score        |  94.6068  |
|  7 | fleschkincaid_score |   1.1608  |
|  8 | gunningfog_score    |   4.03082 |
|  9 | smog_score          |   5.98547 |
| 10 | dalechall_score     |   6.38793 |
+----+---------------------+-----------+


In [88]:
hagrid1 = Textatistic(book_tdf_char["quotes"].loc[12])
hagrid_values1 = hagrid1.dict()
df = pd.DataFrame(list(hagrid_values1.items()), columns= ['metric', 'value'])
print('hagrid', (tabulate(df, headers='keys', tablefmt='psql')))

hagrid +----+---------------------+-------------+
|    | metric              |       value |
|----+---------------------+-------------|
|  0 | char_count          | 1623        |
|  1 | word_count          |  359        |
|  2 | sent_count          |   74        |
|  3 | sybl_count          |  425        |
|  4 | notdalechall_count  |  103        |
|  5 | polysyblword_count  |   11        |
|  6 | flesch_score        |  101.758    |
|  7 | fleschkincaid_score |    0.271386 |
|  8 | gunningfog_score    |    3.16617  |
|  9 | smog_score          |    5.33165  |
| 10 | dalechall_score     |    8.40741  |
+----+---------------------+-------------+


In [89]:
harry1 = Textatistic(book_tdf_char["quotes"].loc[13])
harry_values1 = harry1.dict()
df = pd.DataFrame(list(harry_values1.items()), columns= ['metric', 'value'])
print('harry', (tabulate(df, headers='keys', tablefmt='psql')))

harry +----+---------------------+-------------+
|    | metric              |       value |
|----+---------------------+-------------|
|  0 | char_count          | 1586        |
|  1 | word_count          |  359        |
|  2 | sent_count          |   97        |
|  3 | sybl_count          |  420        |
|  4 | notdalechall_count  |   57        |
|  5 | polysyblword_count  |    9        |
|  6 | flesch_score        |  104.104    |
|  7 | fleschkincaid_score |   -0.341584 |
|  8 | gunningfog_score    |    2.4832   |
|  9 | smog_score          |    4.86922  |
| 10 | dalechall_score     |    6.32712  |
+----+---------------------+-------------+


In [90]:
hermione1 = Textatistic(book_tdf_char["quotes"].loc[14])
hermione_values1 = hermione1.dict()
df = pd.DataFrame(list(hermione_values1.items()), columns= ['metric', 'value'])
print('hermione', (tabulate(df, headers='keys', tablefmt='psql')))

hermione +----+---------------------+-----------+
|    | metric              |     value |
|----+---------------------+-----------|
|  0 | char_count          | 805       |
|  1 | word_count          | 181       |
|  2 | sent_count          |  32       |
|  3 | sybl_count          | 222       |
|  4 | notdalechall_count  |  22       |
|  5 | polysyblword_count  |  10       |
|  6 | flesch_score        |  97.3304  |
|  7 | fleschkincaid_score |   1.08887 |
|  8 | gunningfog_score    |   4.47244 |
|  9 | smog_score          |   6.32262 |
| 10 | dalechall_score     |   5.83628 |
+----+---------------------+-----------+


In [91]:
ron1 = Textatistic(book_tdf_char["quotes"].loc[29])
ron_values1 = ron1.dict()
df = pd.DataFrame(list(ron_values.items()), columns= ['metric', 'value'])
print('ron', (tabulate(df, headers='keys', tablefmt='psql')))

ron +----+---------------------+-------------+
|    | metric              |       value |
|----+---------------------+-------------|
|  0 | char_count          | 4646        |
|  1 | word_count          | 1032        |
|  2 | sent_count          |  261        |
|  3 | sybl_count          | 1207        |
|  4 | notdalechall_count  |  216        |
|  5 | polysyblword_count  |   26        |
|  6 | flesch_score        |  103.876    |
|  7 | fleschkincaid_score |   -0.246962 |
|  8 | gunningfog_score    |    2.58936  |
|  9 | smog_score          |    4.93216  |
| 10 | dalechall_score     |    7.1375   |
+----+---------------------+-------------+


## Limitations

quotes in the book with "he said" and "she said" were not taken, which definitely changes things

Snape was an issue during the vecterizing process (wasn't calculated correctly)

vectorizing was also an issue with they've and they're -- had ve and re as separate words